In [156]:
import functools

import clingo

from typing import Sequence

In [157]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' ', report_answersets=True):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            if report_answersets:
                print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        solve_result = solve_handle.get()
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(solve_result, "{}{}".format(len(models), cardinality_suffix))

    return models
run = functools.partial(solve, report_answersets=False)

In [158]:
def check(programs: Sequence[str], consistency_checks:str, grounding_context=None):
    print("Run without checks:")
    wo_checks = solve(programs, grounding_context, report_answersets=False)
    print("Run with checks:")
    w_checks = solve((*programs, consistency_checks), grounding_context, report_answersets=False)
    return len(wo_checks) == len(w_checks)

In [159]:
gridworld = """

width(3).
height(3).

coordinates((1..W,1..H)) :- width(W), height(H).

adjacent(P1, P2) :-
  coordinates(P1), coordinates(P2),
  P1 = (X1, Y1),
  P2 = (X2, Y2),
  |X1 - X2| + |Y1 - Y2| = 1.

diagonal(P1, P2) :-
  coordinates(P1), coordinates(P2),
  P1 = (X1, Y1),
  P2 = (X2, Y2),
  |X1 - X2| = 1,
  |Y1 - Y2| = 1.

surrounding(P1, P2) :-
  diagonal(P1, P2).
surrounding(P1, P2) :-
  adjacent(P1, P2).

"""
solve([gridworld]);

Answer 1: { coordinates((1,1)) coordinates((1,2)) coordinates((1,3)) coordinates((2,1)) coordinates((2,2)) coordinates((2,3)) coordinates((3,1)) coordinates((3,2)) coordinates((3,3)) height(3) width(3) adjacent((1,1),(1,2)) adjacent((1,1),(2,1)) adjacent((1,2),(1,1)) adjacent((1,2),(1,3)) adjacent((1,2),(2,2)) adjacent((1,3),(1,2)) adjacent((1,3),(2,3)) adjacent((2,1),(1,1)) adjacent((2,1),(2,2)) adjacent((2,1),(3,1)) adjacent((2,2),(1,2)) adjacent((2,2),(2,1)) adjacent((2,2),(2,3)) adjacent((2,2),(3,2)) adjacent((2,3),(1,3)) adjacent((2,3),(2,2)) adjacent((2,3),(3,3)) adjacent((3,1),(2,1)) adjacent((3,1),(3,2)) adjacent((3,2),(2,2)) adjacent((3,2),(3,1)) adjacent((3,2),(3,3)) adjacent((3,3),(2,3)) adjacent((3,3),(3,2)) diagonal((1,1),(2,2)) diagonal((1,2),(2,1)) diagonal((1,2),(2,3)) diagonal((1,3),(2,2)) diagonal((2,1),(1,2)) diagonal((2,1),(3,2)) diagonal((2,2),(1,1)) diagonal((2,2),(1,3)) diagonal((2,2),(3,1)) diagonal((2,2),(3,3)) diagonal((2,3),(1,2)) diagonal((2,3),(3,2)) diagon

In [160]:
actionlanguage_axioms = """

%% E causes F if C
occurs(Who2, change(Property, Value), T) :-
  occurs(Who1, E, T),
  causes_if(Who1, E, Who2, change(Property, Value), CondId),
  not refuted(CondId, T).

%% F if C
occurs(Who, change(Property, Value), T) :-
  time(T),
  if(Who, change(Property, Value), CondId),
  not refuted(CondId, T).

%% E impossible if C
occurs(Who, change(impossible, E), T-1) :-
  time(T),
  impossible_if(Who, E, CondId),
  not refuted(CondId, T).

refuted(false).
:- refuted(true).

observe(Who, What, ST) :-
  init(Who, What),
  time(ST), not time(ST-1).

observe(Who, value(Property, Value), T+1) :-
  time(T+1),
  observe(Who, value(Property, Value), T),
  not occurs(Who, change(Property, _), T).

observe(Who, value(Property, NewValue), T+1) :-
  time(T+1),
  occurs(Who, change(Property, NewValue), T).

"""
run([actionlanguage_axioms]);

SAT 1


<block>:6:3-60: info: atom does not occur in any rule head:
  causes_if(Who1,E,Who2,change(Property,Value),CondId)

<block>:7:7-25: info: atom does not occur in any rule head:
  refuted(CondId,T)

<block>:11:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:12:3-43: info: atom does not occur in any rule head:
  if(Who,change(Property,Value),CondId)

<block>:13:7-25: info: atom does not occur in any rule head:
  refuted(CondId,T)

<block>:17:3-10: info: atom does not occur in any rule head:
  time(T)

<block>:18:3-32: info: atom does not occur in any rule head:
  impossible_if(Who,E,CondId)

<block>:19:7-25: info: atom does not occur in any rule head:
  refuted(CondId,T)

<block>:25:3-18: info: atom does not occur in any rule head:
  init(Who,What)

<block>:26:3-11: info: atom does not occur in any rule head:
  time(ST)

<block>:26:17-27: info: atom does not occur in any rule head:
  time((ST+-1))

<block>:29:3-12: info: atom does not occur in any rule head:
  time((T

In [161]:
domain = """

refuted(at(Agent,(X,Y)), T) :-
  agent(Agent), coordinates((X,Y)), time(T),
  not observe(agent(Agent), value(at, (X,Y)), T).

causes_if(agent(Agent), action(move, up), agent(Agent), change(at, (X,Y+1)), at(Agent, (X,Y))) :-
  agent(Agent),
  coordinates((X,Y+1)),
  coordinates((X,Y)).

causes_if(agent(Agent), action(move, right), agent(Agent), change(at, (X+1,Y)), at(Agent,(X,Y))) :-
  agent(Agent),
  coordinates((X+1,Y)),
  coordinates((X,Y)).

causes_if(agent(Agent), action(move, down), agent(Agent), change(at, (X,Y-1)), at(Agent,(X,Y))) :-
  agent(Agent),
  coordinates((X,Y-1)),
  coordinates((X,Y)).

causes_if(agent(Agent), action(move, left), agent(Agent), change(at, (X-1,Y)), at(Agent,(X,Y))) :-
  agent(Agent),
  coordinates((X-1,Y)),
  coordinates((X,Y)).


"""
run([domain]);

SAT 1


<block>:4:3-15: info: atom does not occur in any rule head:
  agent(Agent)

<block>:4:17-35: info: atom does not occur in any rule head:
  coordinates((X,Y))

<block>:4:37-44: info: atom does not occur in any rule head:
  time(T)

<block>:5:7-49: info: atom does not occur in any rule head:
  observe(agent(Agent),value(at,(X,Y)),T)

<block>:8:3-15: info: atom does not occur in any rule head:
  agent(Agent)

<block>:9:3-23: info: atom does not occur in any rule head:
  coordinates((X,(Y+1)))

<block>:10:3-21: info: atom does not occur in any rule head:
  coordinates((X,Y))

<block>:13:3-15: info: atom does not occur in any rule head:
  agent(Agent)

<block>:14:3-23: info: atom does not occur in any rule head:
  coordinates(((X+1),Y))

<block>:15:3-21: info: atom does not occur in any rule head:
  coordinates((X,Y))

<block>:18:3-15: info: atom does not occur in any rule head:
  agent(Agent)

<block>:19:3-23: info: atom does not occur in any rule head:
  coordinates((X,(Y+-1)))

<block>:2

In [162]:
instance = """

time(1..5).
agent(a).

init(agent(a), value(at, (2,2))).

occurs(agent(a), action(move, up), 1).
occurs(agent(a), action(move, right), 2).
occurs(agent(a), action(move, down), 3).
occurs(agent(a), action(move, left), 4).

"""
run([instance]);

SAT 1


In [163]:
prg = (
    gridworld,
    actionlanguage_axioms,
    domain,
    instance,
)

In [164]:
solve(prg,sep='\n');

Answer 1: {
agent(a)
coordinates((1,1))
coordinates((1,2))
coordinates((1,3))
coordinates((2,1))
coordinates((2,2))
coordinates((2,3))
coordinates((3,1))
coordinates((3,2))
coordinates((3,3))
height(3)
refuted(false)
time(1)
time(2)
time(3)
time(4)
time(5)
width(3)
adjacent((1,1),(1,2))
adjacent((1,1),(2,1))
adjacent((1,2),(1,1))
adjacent((1,2),(1,3))
adjacent((1,2),(2,2))
adjacent((1,3),(1,2))
adjacent((1,3),(2,3))
adjacent((2,1),(1,1))
adjacent((2,1),(2,2))
adjacent((2,1),(3,1))
adjacent((2,2),(1,2))
adjacent((2,2),(2,1))
adjacent((2,2),(2,3))
adjacent((2,2),(3,2))
adjacent((2,3),(1,3))
adjacent((2,3),(2,2))
adjacent((2,3),(3,3))
adjacent((3,1),(2,1))
adjacent((3,1),(3,2))
adjacent((3,2),(2,2))
adjacent((3,2),(3,1))
adjacent((3,2),(3,3))
adjacent((3,3),(2,3))
adjacent((3,3),(3,2))
diagonal((1,1),(2,2))
diagonal((1,2),(2,1))
diagonal((1,2),(2,3))
diagonal((1,3),(2,2))
diagonal((2,1),(1,2))
diagonal((2,1),(3,2))
diagonal((2,2),(1,1))
diagonal((2,2),(1,3))
diagonal((2,2),(3,1))
diagonal

<block>:12:3-43: info: atom does not occur in any rule head:
  if(Who,change(Property,Value),CondId)

<block>:18:3-32: info: atom does not occur in any rule head:
  impossible_if(Who,E,CondId)

